In [ ]:
!pip install numpy
!pip install qutip
!pip install cirq

The following code deals with the creation of $3 \otimes 3$ separable states and corresponding random $9 \times 9$ density matrices.

In [4]:
import qutip
import numpy as np
import random
from matplotlib import pyplot as plt
from cirq.linalg import is_unitary
from cirq.qis.states import validate_density_matrix

"""
We are trying to create separable states of order 3 \otimes 3.
Procedure:
- Create random density matrices of size 3x3.
- Take random sum of tensor product of random a, b \in 3x3 dm-set
with a certain random probability associated with it.
"""


def embed(matrix: np.ndarray) -> list:
    x, y = matrix.shape

    # Validate the given density matrix
    try:
        validate_density_matrix(matrix, qid_shape=x)
    except:
        print(f"Not valid density matrix: {matrix}")
        return
    matrix = matrix.tolist()

    # Flatten and embed the density matrix by taking care that,
    # it has (n - 1) diagonal values and upper traingular matrix.
    flattened = []
    for i in range(x):
        for j in range(y):
            if i < j:
                flattened.append(matrix[i][j])

    # Embed the real and complex values respectively.
    embedding = []
    for i in flattened:
        embedding.extend([i.real, i.imag])

    # Embed diagonal matrices
    for i in range(1, x):
        embedding.append(matrix[i][i].real)

    return embedding


def get_separable_state(n: int, m: int, max_len: int = 50) -> np.ndarray:
    l = random.randint(1, max_len)
    prob = np.array([random.random() for _ in range(l)])
    prob = prob / np.sum(prob)

    separable_state = np.zeros((n * m, n * m))
    for i in range(l):
        tensor = np.kron(np.array(qutip.rand_dm(n)), np.array(qutip.rand_dm(m)))
        separable_state = np.add(separable_state, prob[i] * tensor)

    # print(separable_state, "\n\n", separable_state.shape, "\n\n")
    return separable_state


def get_separable_states(num: int, n: int, m: int) -> np.ndarray:
    separable_states = []
    for i in range(num):
        separable_states.append(embed(get_separable_state(n, m)))
        # print(i)
    return np.array(separable_states)


def get_random_states(num: int, n: int, m: int) -> np.ndarray:
    random_states = []
    for i in range(num):
        random_states.append(embed(np.array(qutip.rand_dm(n * m))))
        # print(i)
    return np.array(random_states)

In [ ]:
!pip install pyod

In [5]:
# import libraries
from pyod.utils.data import generate_data
from pyod.models.ocsvm import OCSVM
from pyod.utils.example import visualize


def accuracy(a, b):
    n = range(len(a))
    return sum([a[i] != b[i] for i in n])


# X_train, X_test, y_train, y_test = generate_data(
#     n_train=5, n_test=1, n_features=3, behaviour="new"
# )


def load_data(n: int, m: int):
    X_train = np.load("/content/train_data.npy")
    y_train = [0 for _ in range(50000)]

    X_test = np.concatenate(
        (np.load("/content/test_data.npy")[:50], get_separable_states(50, 3, 3))
    )
    y_test = [0 for _ in range(100)]
    return X_train, X_test, y_train, y_test

In [7]:
n = 3
m = 3
X_train, X_test, y_train, y_test = load_data(n, m)
clf = OCSVM(nu=0.9, gamma=0.5, kernel="rbf", verbose=1)
clf.fit(X_train)

[LibSVM]

OCSVM(cache_size=200, coef0=0.0, contamination=0.1, degree=3, gamma=0.5,
   kernel='rbf', max_iter=-1, nu=0.9, shrinking=True, tol=0.001, verbose=1)

In [8]:
y_train_pred = clf.predict(X_train)

In [9]:
y_test_pred = clf.predict(X_test)

In [10]:
print(y_test_pred)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [11]:
print(y_test_pred[:50])

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [12]:
print(y_test_pred[50:])

[1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]


Let's try out a few well known test cases.


1. Maximally mixed matrix
2.
3.


How to try out tests?


*   Create your separable or entangled matrix.
*   List item




In [19]:
# Maximally mixed matrix
I = (1/9) * np.eye(9)
I = embed(I)
if clf.predict(np.array([I]))[0] == 0:
    print("Separable, correct!")

Separable, correct!


In [ ]:
# Random matrices
for i in range(int(1e8)):
    # print(f"Test {i + 1}")
    rho = qutip.rand_dm(9)
    rho.dims = [[3, 3], [3, 3]]
    rho_out = qutip.partial_transpose(rho, [0, 1])
    if min(rho_out.eigenenergies()) >= 0:
        # print(min(rho_out.eigenenergies()))
        M = np.array(rho)
        M = embed(M)
        print(clf.predict(np.array([M]))[0])

1
1
1
0
1
1
1
0
1
0
1
1
1
1
1
1
1
0
1
1
1
1
1
0
1
1
1
0
1
1
1
1
1
1
0
0
1
1
1
1
0
1
1
0
1
1
1
1
1
1
1
0
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
0
1
1
1
1
1
1
1
1
0
1
1
1
